In [ ]:
import os
from pathlib import Path
from typing import Final
import tensorflow as tf
import numpy as np
import cv2
import utils

In [ ]:
EPOCHS: Final[int] = 4
ACCURACY: Final[int] = 97.31

In [ ]:
model = tf.keras.models.load_model(os.path.join(
    utils.DATA_DIR, 
    "models", 
    str(EPOCHS), 
    f"mnist-{EPOCHS}-{ACCURACY}.model"
))

In [ ]:
test_images_paths: list[Path] = [
    os.path.join(utils.TEST_DIR, f) 
    for f in os.listdir(utils.TEST_DIR) 
    if f.endswith(".png")
]

In [ ]:
utils.log_to_file(f"Starting testing model with EPOCHS: {EPOCHS} and ACCURACY: {ACCURACY}")

num_faults: int = 0
for image_path in test_images_paths:
    image: np.ndarray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_array: np.ndarray = np.invert(np.array([image]))
    predected: np.ndarray = model.predict(image_array)
    image_name: str = f"Predicted: {np.argmax(predected)}"
    # show_image(image, image_name)
    utils.log_to_file(f"predected: {np.argmax(predected)} and it was {image_path.split('/')[-1].split('.')[0]}")
    if np.argmax(predected) != int(image_path.split('/')[-1].split('.')[0]):
        num_faults += 1

utils.log_to_file(f"Accuracy: {100 - (num_faults / len(test_images_paths)) * 100}%")
utils.log_to_file(f"Number of faults: {num_faults:,}/{len(test_images_paths):,}")
utils.log_to_file(f"Number of correct: {(len(test_images_paths) - num_faults):,}/{len(test_images_paths):,}") 

#### Confusion Matrix

In [ ]:
test_images_flattened: np.ndarray[np.ndarray] = np.array([
    np.invert(np.array([
        cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    ]))
    for image_path in test_images_paths
])

test_images_predected: np.ndarray[np.ndarray] = model.predict(test_images_flattened)

test_images_predected_labels: list[int] = [
    np.argmax(prediction)
    for prediction in test_images_predected
]

true_labels: list[int] = [
    int(image_path.split('/')[-1].split('.')[0])
    for image_path in test_images_paths
]

In [ ]:
confusion_matrix = tf.math.confusion_matrix(
    labels=true_labels, 
    predictions=test_images_predected_labels
)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(
    confusion_matrix.numpy(),
    index=list("0123456789"),
    columns=list("0123456789"),
)

plt.figure(figsize=(10,7))
sn.heatmap(df_cm, annot=True)
Path(os.path.join(
    utils.DATA_DIR, 
    "confusion-matrices", 
    str(EPOCHS), 
    "test"
)).mkdir(parents=True, exist_ok=True)

confusion_matrix_path: Path = os.path.join(
    utils.DATA_DIR, 
    "confusion-matrices", 
    str(EPOCHS),
    "test",
    f"mnist-{EPOCHS}-{ACCURACY}.png"
)
plt.savefig(confusion_matrix_path)
plt.show()

utils.log_to_file(f"Confusion matrix saved to: '{str(confusion_matrix_path)}'")